In [1]:
# !pip install tqdm
# !pip install torch
# !pip install pygments
# !pip install google_play_scraper

In [2]:
from tqdm import tqdm
import pandas as pd
import seaborn as sns
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter
from google_play_scraper import Sort,reviews, app
import json

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [8]:
apps_ids = [
    'br.com.brainweb.ifood',
    'com.cerveceriamodelo.modelonow',
    'com.mcdo.mcdonalds',
    'habibs.alphacode.com.br',
    'com.xiaojukeji.didi.brazil.customer',
    'com.ubercab.eats',
    'com.grability.rappi',
    'burgerking.com.br.appandroid', 
    'com.vanuatu.aiqfome'
]

In [9]:
app_infos = []

for ap in tqdm(apps_ids):
    info = app(ap, lang='en', country='us')
    del info['comments']
    app_infos.append(info)

100%|██████████| 9/9 [00:06<00:00,  1.30it/s]


In [12]:
app_infos_df.shape

(9, 44)

In [13]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.head()

,title,description,descriptionHTML,summary,installs,minInstalls,realInstalls,score,ratings,reviews,...,contentRatingDescription,adSupported,containsAds,released,updated,version,recentChanges,recentChangesHTML,appId,url
0,iFood comida e mercado em casa,"Compre em <b>mercados, restaurantes, farmácias...","Compre em <b>mercados, restaurantes, farmácias...","Order food delivery, market, drinks, pharmacy ...","50,000,000+",50000000,93859987,4.700685,10405516,2495,...,None,False,False,"Apr 25, 2012",1667838595,Varies with device,"Olá, amante de comida gostosa a qualquer momen...","Olá, amante de comida gostosa a qualquer momen...",br.com.brainweb.ifood,https://play.google.com/store/apps/details?id=...
1,Zé Delivery de Bebidas,Zé Delivery: o maior app de bebidas do país! \...,Zé Delivery: o maior app de bebidas do país! <...,"Delivery of drinks at a low price, with fast d...","10,000,000+",10000000,15628086,4.445545,757341,240,...,None,False,False,"Aug 19, 2016",1665754952,22.41.1,"Olha, ooolha: o carro da atualização do app ch...","Olha, ooolha: o carro da atualização do app ch...",com.cerveceriamodelo.modelonow,https://play.google.com/store/apps/details?id=...
2,McDonald's Offers and Delivery,Enter the new McDonald's App and get exclusive...,Enter the new McDonald&#39;s App and get exclu...,"Enjoy esclusive discounts, promotions and coup...","50,000,000+",50000000,56951250,4.503673,819119,2813,...,None,False,False,"Mar 27, 2017",1666685709,3.17.0,Performance improvements and bug fixes,Performance improvements and bug fixes,com.mcdo.mcdonalds,https://play.google.com/store/apps/details?id=...
3,Habib's,Novo App Habib’s \r\n+ Ágil \r\n+ Moderno \r\n...,Novo App Habib’s <br>+ Ágil <br>+ Moderno <br>...,Get to know the new Habib's application and sa...,"5,000,000+",5000000,6555665,3.722222,180672,20,...,None,False,False,None,1667517085,3.9.4,None,None,habibs.alphacode.com.br,https://play.google.com/store/apps/details?id=...
4,99 Food – Food Delivery,Did you feel hungry? Open the app!\r\n99Food c...,Did you feel hungry? Open the app!<br>99Food c...,Your favorite food in one app,"5,000,000+",5000000,6638668,3.894737,105164,7,...,None,True,True,None,1667992348,2.0.16,1. Optimized visual effects to enhance your us...,1. Optimized visual effects to enhance your us...,com.xiaojukeji.didi.brazil.customer,https://play.google.com/store/apps/details?id=...


In [18]:
app_reviews = []

for ap in tqdm(apps_ids):
    for score in list(range(1,6)):
        for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
            rvs, _ = reviews(
                ap,
                lang='pt',
                country='br',
                sort=sort_order,
                count = 200 if score == 3 else 100,
                filter_score_with = score
            )
            for r in rvs:
                r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
                r['appId'] = ap
            app_reviews.extend(rvs)

100%|██████████| 9/9 [00:55<00:00,  6.20s/it]


In [19]:
len(app_reviews)

10800

In [20]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,69577120-91aa-49a5-bee6-5f92bbb5b172,Nicholas Zein,https://play-lh.googleusercontent.com/a-/ACNPE...,Nunca sou de fazer reviews. Mas tive q fazer e...,1,156,9.178.0,2022-10-30 17:22:25,None,NaT,most_relevant,br.com.brainweb.ifood
1,a3916326-cf04-4943-a185-4230a53b5413,Gustavo Colombini,https://play-lh.googleusercontent.com/a-/ACNPE...,"Ótimo para pedir comida rapidamente, boas prom...",1,862,9.169.0,2022-08-26 13:47:41,None,NaT,most_relevant,br.com.brainweb.ifood
2,368bd383-b417-4243-ad29-e67910e27038,Fernando Jardim,https://play-lh.googleusercontent.com/a-/ACNPE...,Tive uma experiência desagradável com o app e ...,1,371,9.161.0,2022-07-06 19:57:05,None,NaT,most_relevant,br.com.brainweb.ifood
3,f1dcbbdd-b3af-46df-a914-55baf9054e03,Nadia Nunes,https://play-lh.googleusercontent.com/a-/ACNPE...,O app é bom em relação a escolha de restaurant...,1,2085,9.164.0,2022-07-19 22:19:34,None,NaT,most_relevant,br.com.brainweb.ifood
4,99a38c03-a02a-461a-b97b-564869c235f3,Claudia Regina Garcia,https://play-lh.googleusercontent.com/a/ALm5wu...,"O app em si é bom. Entretanto, há poucas forma...",1,575,9.165.0,2022-08-01 08:22:56,None,NaT,most_relevant,br.com.brainweb.ifood


In [21]:
app_reviews_df.to_csv('../Datasets/Reviews.csv', index=None, header=True)